## Markov Decision Process and Q-value iteration

## MDP specified by: 
 * ### state and action spaces
 * ### state-action-state' transition probabilities (0 indicates no link in Markov chain)
 * ### state-action-state' rewards
 


In [ ]:
import numpy as np
from tqdm import tqdm

In [9]:
transition_probabilities = [ 
# shape=[s, a, s']
# So dims #S * #A * #S
[[0.7, 0.3, 0.0], 
 [1.0, 0.0, 0.0], 
 [0.8, 0.2, 0.0]],
    
 [[0.0, 1.0, 0.0], 
  None, 
  [0.0, 0.0, 1.0]],

 [None, 
  [0.8, 0.1, 0.1], 
  None]
]

rewards = [ # shape=[s, a, s']
[[+10, 0, 0], # Self loop: +10 for selecting action 0 in state 0 and returning to state 0
 [0, 0, 0], 
 [0, 0, 0]],

[[0, 0, 0], 
 [0, 0, 0], 
 [0, 0, -50]],

[[0, 0, 0], 
 [+40, 0, 0], 
 [0, 0, 0]]
]




## The set of actions is fixed and the same for each state, so there is a global actions enumeration

In [5]:
possible_actions = [ # shape = [s, a] 
 [0, 1, 2], 
 [0, 2], 
 [1]]

# Tabular Value Estimation: Q-value Iteration

## Initialize Q-values

In [6]:
Q_values = np.full((3, 3), -np.inf)

In [7]:
for state, actions in enumerate(possible_actions):
    Q_values[state, actions] = 0.0 # Smart indexing for assigning values at once

In [8]:
gamma = 0.90

In [11]:
for iteration in range(50):
    Q_prev  = Q_values.copy()
    for s in range(3): # 3 states in MDP
        for a in possible_actions[s]:
            Q_values[s, a] = np.sum([
                transition_probabilities[s][a][sp] * (rewards[s][a][sp] + gamma * Q_prev[sp].max())
                for sp in range(3)
            ])

In [12]:
Q_values

array([[18.91891892, 17.02702702, 13.62162162],
       [ 0.        ,        -inf, -4.87971488],
       [       -inf, 50.13365013,        -inf]])

## Optimal policy: greedy policy over Q*

In [13]:
Q_values.argmax(axis=1)

array([0, 0, 1])

# Tabular Q-learning: TD for Q values, i.e. action-value estimates

## Q-learning is an off-policy learning, learned Q_values table is in principle not used for actions selection, but can be used in e.g. $\epsilon$-greedy policy. In any case data samples are not generated by the same object that is being learned

## Sort of brute force learning or bare statistics: learning happens even with random policy if the whole space is explored
## Every *state-action-state'* transiton should be tried at least once to know the rewards (if rewarding is deterministic)
## Every *state-action* choice should be tried many times to estimate *state-action(-state')* transition probabilities

In [15]:
# Function performs action and performs state change based on 
# prescribed states transition probabilities
def step(state, action):
    probas = transition_probabilities[state][action]
    next_state = np.random.choice([0, 1, 2], p=probas)
    reward = rewards[state][action][next_state]
    return next_state, reward

## Random exploratory policy

In [16]:
def exploration_policy(state):
    return np.random.choice(possible_actions[state])

## Initialize Q-values and hyperparameters

In [17]:
Q_values = np.full((3, 3), -np.inf)

In [18]:
alpha0 = 0.05 # Initial learning rate
decay = 0.005 # LR decay
gamma = 0.90 # Discount factor

## General SARS paradigm like in Barto Sutton

## Note: max in Q_values is in fact a trace of greedy policy selecting values based on best possible action

In [24]:
state = 0 # Initial state

for iteration in tqdm(range(10_000)):
    # Take action
    action = exploration_policy(state)
    # Observe Reward, Next State
    next_state, reward = step(state, action)
    # Greedy policy next state value for max in Q-learning
    next_value = Q_values[next_state].max()
    # LR schedule - mandatory for SGD-like convergence
    alpha = alpha0 / (1 + iteration * decay)
    # Q-learning iteration step
    Q_values[state, action] *= 1 - alpha
    Q_values[state, action] += alpha * (reward + gamma * next_value)
    # State progression
    state = next_state
    

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 32109.77it/s]


## One can enhance exploration policy by counting how many times actions were selected in states and introducing curiosity with exploration function 
## In this function actions more often tried are assigned less extra value
## In infinite exploration limit cuirosity decays to zero and Q_values converge to pure values

$$ Q(s, a) \underset{\alpha}\leftarrow f(Q(s', a'), N(s', a'))$$
$$f(Q, N) = Q + \frac{\kappa}{1+N}$$